In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

from src.utils import evaluate_retriever
from src.utils import RetrieverMetrics, save_retriever_trial_log, prepare_rerankretriever_configs, load_benchmarks_df

from src.Retriever import RerankRetriever, RerankRetrieverConfig
from src.Reader import LLM_Model, LLM_Config

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !!! TO CHANGE !!!
TRIAL = 2
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

# Retriever part
RETRIEVER_PARAMS = {
    "stage1_retriever_config": {
        "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
        "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
        "model_kwargs": {'device':'cuda'},
        "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
        "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 10}},
    "scorer_config": {
        "system_prompt": 'Ты - экспертная система, задача которой - оценить релевантность контекста по отношению к данному вопросу. На вход поступает вопрос и контекст в текстовом формате, твоя задача - вернуть одну из двух меток: "{cert_label}" - в контексте содержится информация, которая может быть использована для верного ответа на данный вопрос и "{uncert_label}" - в контексте не содержится информации, которая может быть использована для верного ответа на данный вопрос. Не пиши ничего другого, кроме предсказываемой метки релевантности: "{cert_label}" либо "{uncert_label}". \n\nПримеры: \n\nВопрос: По просьбе Мадонны поменяли аранжировщика на более опытного?\nКонтекст: [\' Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].\']\nМетка: {cert_label}\n\nВопрос: Для чего осуществляется деятельность официальных органов по получению и применению финансовых средств?\nКонтекст: [\' Общественные финансы связываются с деятельностью государства (или местной власти), заключающейся в получении и применении средств, необходимых для выполнения надлежащих функций[51], с процессом и механизмом формирования и использования государственных ресурсов, балансом доходов и расходов, соответствующим административным контролем[3]. Некоторые авторы[52] также подчёркивают, что предмет общественных финансов лежит на границе между экономикой и политикой, а также отмечают, что в современных обществах доходы и расходы органов государственной власти состоят, почти исключительно, из денежных поступлений.\']\nМетка: {cert_label}\n\nВопрос: Кто запретил Нансену высадиться на побережье Гренландии?\nКонтекст: [\'29 февраля 1892 года столетие со дня рождения Россини было отмечено должным образом во Флоренции, но горожане, любившие его музыку, ощущали неловкость оттого, что на могиле Россини в Санта-Кроче все ещё не было подходящего памятника. На первом открытом конкурсе, состоявшемся в 1897 году, комиссия не сочла ни один проект приемлемым. На втором конкурсе, который состоялся в следующем году, новая комиссия одобрила проект Джузеппе Кассиоли. Получившийся в результате монумент был открыт 23 июня 1902 года. Поставленный на фоне северной стены Санта-Кроче, он содержит надпись: Джоаккино Россини . У его основания три датированных картуша гласят: Пезаро , Флоренция , Париж , таким образом, официально отрицая справедливые притязания Болоньи на то, что она внесла свою значительную лепту в длительную славу Россини.\']\nМетка: {uncert_label}\n\nВопрос: Кто в 1536 году был губернатором провинции Санта-Марта?\nКонтекст: [\'9 (21) мая владыка Иннокентий заложил в Усть-Зейской станице храм в честь Благовещения Пресвятой Богородицы (по имени этого храма был впоследствии назван город). 12 (24) мая начались переговоры между представителями Цинской империи и чиновниками российского внешнеполитического ведомства во главе с Н. Н. Муравьёвым. Переговоры велись 5 дней, сопровождаясь многочисленными церемониями по китайским и русским обычаям. 16 (28) мая 1858 года был подписан Айгунский договор, по условиям которого весь левый берег Амура признавался российским.\']\nМетка: {uncert_label}',
    "user_prompt_template": "Вопрос: {q}\nКонтекст: {c}\nМетка: ",
    "cert_label": "1",
    "uncert_label": "0",
    "gen": {'max_new_tokens': 20, 'eos_token_id': 79097},
    "batch_size": 8
    }
}

ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}

# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

In [3]:
retrivers_config, benchmarks_path = prepare_rerankretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
reader_config = LLM_Config()
metrics = RetrieverMetrics()

In [4]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

In [5]:
READER = LLM_Model(reader_config)

Loading checkpoint shards:  50%|█████     | 2/4 [03:08<03:08, 94.05s/it]


KeyboardInterrupt: 

In [ ]:
RETRIEVERS = {name: RerankRetriever(retrivers_config[name], READER) for name in benchmarks_df.keys()}

In [ ]:
retriever_scores, retriever_cache, _ = evaluate_retriever(benchmarks_df, RETRIEVERS, metrics, show_step=10,
                                                          topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

In [ ]:
save_retriever_trial_log(SAVE_LOGDIR, retriever_scores, SAVE_HYPERPARAMS, SAVE_RETRIEVERCACHE,
                         retriever_cache, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE, RETRIEVER_PARAMS, ADDITIONAL_PARAMS)

In [ ]:
import gc
import torch

torch.cuda.empty_cache()
gc.collect()